In [ ]:
from functools import reduce
import math
import random

In [30]:
not_use_pointer = 0
factorys_number = 0
factorys = []
customers_number = 0
customers = []
T = 10000
data = None

In [64]:
def start(num):
    global not_use_pointer
    global factorys_number
    global factorys
    global customers_number
    global customers
    global T
    global data
    with open("Instances/p"+str(num),'r') as f:
        data = f.read().split()
        #print(data)
        for x in range(len(data)):
            if len(data[x])>10:
                data = data[:x]
                break
        def map_f(x):
            if x[-1]=='.':
                return int(x[:-1])
            else:
                return int(x)
        data = list(map(map_f,data))
    not_use_pointer = 0
    factorys_number = data[not_use_pointer]
    factorys = []
    not_use_pointer += 1
    customers_number = data[not_use_pointer]
    customers = []
    not_use_pointer += 1
    T = 10000
    init()

In [10]:
class Factoray:
    def __init__(self,capacity,open_cost,number):
        self.capacity = capacity
        self.open_cost = open_cost
        self.current_load = 0
        self.number = number

In [11]:
class Customer:
    def __init__(self,load,distance_list,number):
        self.load = load
        self.distance_list = distance_list
        self.assigned_factory = None
    def assign(self,_factory):
        if _factory.capacity - _factory.current_load < self.load:
            return False
        else:
            _factory.current_load+=self.load
            self.assigned_factory = _factory.number
            return True
    def unassign(self):
        if self.assigned_factory != None:
            factorys[self.assigned_factory].current_load -= self.load
            self.assigned_factory = None
            return True
        return False

In [59]:
def init():
    global not_use_pointer
    global factorys_number
    global customers_number
    global factorys
    global customers
    global data
    for x in range(factorys_number):
        #print(not_use_pointer)
        factorys.append(Factoray(data[not_use_pointer],data[not_use_pointer+1],x))
        not_use_pointer += 2
    for x in range(customers_number):
        customers.append(Customer(data[not_use_pointer],[],x))
        not_use_pointer += 1
    for x in range(customers_number):
        for y in range(factorys_number):
            customers[x].distance_list.append(data[not_use_pointer])
            not_use_pointer+=1
    #print(not_use_pointer)
            #print(not_use_pointer)

In [27]:
def judge():
    global factorys
    global customers
    score = 0
    for factory in factorys:
        if factory.current_load != 0:
            score+=factory.open_cost
    for customer in customers:
        score+=customer.distance_list[customer.assigned_factory]
    return score

In [14]:
def find_n_least_big_index(arr,n):
    current_big = -1
    current_index_list = []
    for x in range(n):
        inner_big = 1000000
        inner_index = -1
        for y in range(len(arr)):
            if y not in current_index_list:
                if arr[y] < inner_big:
                    inner_big = arr[y]
                    inner_index = y
        current_index_list.append(inner_index)
        if x == n-1:
            return inner_index

In [68]:
def gridy():
    global customers
    #print(customers)
    for customer in customers:
        for x in range(len(customer.distance_list)):
            target_index = find_n_least_big_index(customer.distance_list,x+1)
            if factorys[target_index].capacity - factorys[target_index].current_load < customer.load:
                continue
            else:
                customer.assign(factorys[target_index])
                break

In [16]:
def possibility(energy):
    global T
    return 1/(1+math.e**(-energy/T))

In [17]:
def sa():
    global T
    while T>10**(-8):
        T = T*0.995
        for x in range(10):
            print(judge())
            change_customer_index = random.randint(0,len(customers)-1)
            #temp = judge()
            while True:
                ## move lots of customers at the same time
                #move_n_customers()
                change_to_factory_index = random.randint(0,len(factorys)-1)
                if factorys[change_to_factory_index].capacity - factorys[change_to_factory_index].current_load >= customers[change_customer_index].load:
                    #compare which one better
                    energy = 0
                    if factorys[customers[change_customer_index].assigned_factory].current_load == customers[change_customer_index].load:
                        energy -= customers[change_customer_index].load
                    energy += customers[change_customer_index].distance_list[change_to_factory_index]
                    energy -= customers[change_customer_index].distance_list[customers[change_customer_index].assigned_factory]
                    if energy<0 or possibility(energy) > random.random():
                        customers[change_customer_index].unassign()
                        customers[change_customer_index].assign(factorys[change_to_factory_index])
                    break

In [65]:
start(1)

['10', '50', '258', '366', '128', '314', '296', '504', '225', '504', '310', '580', '373', '415', '329', '456', '179', '549', '114', '310', '321', '316', '49.', '29.', '23.', '40.', '18.', '17.', '48.', '13.', '15.', '31.', '16.', '31.', '14.', '44.', '23.', '39.', '33.', '45.', '40.', '43.', '38.', '26.', '47.', '34.', '24.', '32.', '28.', '26.', '20.', '13.', '27.', '22.', '14.', '42.', '31.', '37.', '45.', '20.', '38.', '21.', '25.', '13.', '19.', '42.', '10.', '21.', '37.', '27.', '17.', '49.', '260.', '239.', '418.', '557.', '346.', '161.', '288.', '568.', '379.', '392.', '672.', '268.', '122.', '295.', '344.', '143.', '393.', '29.', '656.', '654.', '235.', '329.', '640.', '517.', '329.', '524.', '318.', '516.', '79.', '414.', '458.', '758.', '28.', '167.', '504.', '502.', '475.', '226.', '94.', '422.', '303.', '139.', '362.', '519.', '494.', '88.', '468.', '72.', '370.', '68.', '580.', '484.', '34.', '386.', '180.', '522.', '338.', '636.', '523.', '28.', '646.', '652.', '319.', '4

In [66]:
gridy()

[<__main__.Customer object at 0x0000019954376668>, <__main__.Customer object at 0x0000019954376F98>, <__main__.Customer object at 0x0000019954376BE0>, <__main__.Customer object at 0x0000019954376198>, <__main__.Customer object at 0x0000019954376C88>, <__main__.Customer object at 0x0000019954376160>, <__main__.Customer object at 0x0000019954376320>, <__main__.Customer object at 0x0000019954376400>, <__main__.Customer object at 0x00000199543760F0>, <__main__.Customer object at 0x00000199543769E8>, <__main__.Customer object at 0x00000199543769B0>, <__main__.Customer object at 0x0000019954376B38>, <__main__.Customer object at 0x0000019954376710>, <__main__.Customer object at 0x00000199543762B0>, <__main__.Customer object at 0x0000019954376DD8>, <__main__.Customer object at 0x0000019954376828>, <__main__.Customer object at 0x0000019954376748>, <__main__.Customer object at 0x0000019954376208>, <__main__.Customer object at 0x0000019954376AC8>, <__main__.Customer object at 0x0000019954376C18>,

In [67]:
judge()

10355